In [ ]:
### Import Dependencies

In [173]:
import pandas as pd
import statsapi
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import ssl

In [179]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
#Define url to pull WAR leaders
espn_url = "http://www.espn.com/mlb/war/leaders/_/type/seasonal/year/20"

In [9]:
#define list of stat years to pull and create empty list of DF's to store results
year_list = ['19', '18', '17', '16', '15']
df_list = []

In [10]:
#scrape stat leaders
for year in year_list:
    query_url = espn_url+year
    qdfl = pd.read_html(query_url)
    qdfl[0]['season'] = year
    df_list.append(qdfl[0])

In [14]:
#concat dfs into one
data = pd.concat(df_list)
data.head()

,0,1,2,3,4,5,6,7,8,9,season
0,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,19
1,RK,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,1,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,2,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,3,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [15]:
cols = ['rank', 'player', 'WAR', 'OFF', 'DEF', 'WAA', 'TRPG', 'ORPG', 'RAA', 'WAAWP', 'Season']

In [16]:
#rename columns and drop rank column
data.columns = cols
del data['rank']
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
0,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,19
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [17]:
#drop NAs
data = data.dropna(subset=['player'])

In [18]:
#drop no data holding columns
data = data[~data.player.str.contains("WAR")]

In [19]:
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19


In [20]:
# new data frame with split value columns 
new = data["player"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
data["First Name"]= new[0] 
data["Last Name"]= new[1] 
  
#reset index
data = data.drop(data.index[0])
data.reset_index()
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19,Cody,Bellinger
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19,Alex,Bregman
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19,Mike,Trout
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19,Marcus,Semien
6,Justin Verlander,7.8,0,0,5.8,5,3.4,56.6,0.7,19,Justin,Verlander


In [21]:
#drop duplicate entries
data.drop_duplicates(subset ="player", keep ='first', inplace = True)

In [23]:

#read in master list, change encoding to make it readable
id_list = pd.read_csv('master_id_list.csv', encoding='latin-1')
id_df = pd.DataFrame(id_list)[['mlb_name','espn_id','espn_name', 'espn_pos']].dropna()


In [24]:
#merge ids and player data
new_data = pd.merge(data, id_df,  how='left', left_on=['player'], right_on = ['mlb_name'])

In [25]:
#do not overwrite this csv!!!
#new_data.to_csv('player_list.csv')

In [25]:
clean_data = pd.read_csv('player_list.csv')

In [26]:
#create empyty list for scraping query urls
q_urls=[]

In [27]:
#build query urls
for index, row in clean_data.iterrows():
    espn_tag = row['First Name']+ '-' +row['Last Name']
    espn_tag = espn_tag.lower()
    turl = 'http://www.espn.com/mlb/player/stats/_/id/' + str(row['espn_id']) + '/'+ espn_tag
    q_urls.append(turl)

In [28]:
#add to DF
clean_data['qurl'] = q_urls

In [29]:
#set index to player
clean_data = clean_data.set_index('player')

In [30]:
clean_data.head(3)

,Unnamed: 0,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name,mlb_name,espn_id,espn_name,espn_pos,qurl
player,,,,,,,,,,,,,,,,,
Cody Bellinger,0,9.0,6.6,1.9,7.0,5.0,4.7,72.9,0.5,19,Cody,Bellinger,Cody Bellinger,33912,Cody Bellinger,CF,http://www.espn.com/mlb/player/stats/_/id/3391...
Alex Bregman,1,8.4,7.7,1.2,6.1,5.3,5.0,62.0,0.5,19,Alex,Bregman,Alex Bregman,34886,Alex Bregman,3B,http://www.espn.com/mlb/player/stats/_/id/3488...
Mike Trout,2,8.3,8.3,0.3,6.3,5.5,5.0,65.7,0.5,19,Mike,Trout,Mike Trout,30836,Mike Trout,CF,http://www.espn.com/mlb/player/stats/_/id/3083...


In [31]:
#drop old index column
clean_data = clean_data.drop(columns=['Unnamed: 0'])

In [32]:
clean_data.head(3)

,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name,mlb_name,espn_id,espn_name,espn_pos,qurl
player,,,,,,,,,,,,,,,,
Cody Bellinger,9.0,6.6,1.9,7.0,5.0,4.7,72.9,0.5,19,Cody,Bellinger,Cody Bellinger,33912,Cody Bellinger,CF,http://www.espn.com/mlb/player/stats/_/id/3391...
Alex Bregman,8.4,7.7,1.2,6.1,5.3,5.0,62.0,0.5,19,Alex,Bregman,Alex Bregman,34886,Alex Bregman,3B,http://www.espn.com/mlb/player/stats/_/id/3488...
Mike Trout,8.3,8.3,0.3,6.3,5.5,5.0,65.7,0.5,19,Mike,Trout,Mike Trout,30836,Mike Trout,CF,http://www.espn.com/mlb/player/stats/_/id/3083...


In [95]:
some_df = pd.DataFrame()

In [96]:
#scrape stats from ESPN, clean stats DF, and convert to a Mongo-ready list of dictionaries

for i in clean_data.index:
    temp_df = pd.read_html(clean_data['qurl'][i])
    #above returns a list of dfs, this line pulls the df that contains the stat table
    stat_df = temp_df[1]
    #dropping the blank row at the top and the summary row at the bottom
    stat_df.drop(stat_df.tail(1).index,inplace=True)
    stat_df.drop(stat_df.head(1).index,inplace=True)
    stat_df.columns = stat_df.iloc[0]
    stat_df = stat_df.drop(stat_df.index[0])
    stat_df['playerName'] = clean_data['espn_name'][i]
    stat_df['espnID'] = clean_data['espn_id'][i]
    some_df = pd.concat([some_df, stat_df])
#     , ignore_index=True

/Users/saraflores/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [97]:
some_df.reset_index()

,index,2B,3B,AB,AVG,BB,BLSV,CG,CS,ER,...,SLG,SO,SV,TEAM,W,WAR,WHIP,YEAR,espnID,playerName
0,2,26,4,480,.267,64,NaN,NaN,3,NaN,...,.581,146,NaN,LAD,NaN,4.2,NaN,2017,33912,Cody Bellinger
1,3,28,7,557,.260,69,NaN,NaN,1,NaN,...,.470,151,NaN,LAD,NaN,4.2,NaN,2018,33912,Cody Bellinger
2,4,34,3,558,.305,95,NaN,NaN,5,NaN,...,.629,108,NaN,LAD,NaN,--,NaN,2019,33912,Cody Bellinger
3,5,88,14,1595,.278,228,NaN,NaN,9,NaN,...,.559,405,NaN,Total,NaN,--,NaN,Total,33912,Cody Bellinger
4,2,13,3,201,.264,15,NaN,NaN,0,NaN,...,.478,52,NaN,HOU,NaN,1.8,NaN,2016,34886,Alex Bregman
5,3,39,5,556,.284,55,NaN,NaN,5,NaN,...,.475,97,NaN,HOU,NaN,4.1,NaN,2017,34886,Alex Bregman
6,4,51,1,594,.286,96,NaN,NaN,4,NaN,...,.532,85,NaN,HOU,NaN,6.9,NaN,2018,34886,Alex Bregman
7,5,37,2,554,.296,119,NaN,NaN,1,NaN,...,.592,83,NaN,HOU,NaN,--,NaN,2019,34886,Alex Bregman
8,6,140,11,1905,.286,285,NaN,NaN,10,NaN,...,.527,317,NaN,Total,NaN,--,NaN,Total,34886,Alex Bregman
9,2,6,0,123,.220,9,NaN,NaN,0,NaN,...,.390,30,NaN,LAA,NaN,0.6,NaN,2011,30836,Mike Trout


In [100]:
batters_df = some_df.dropna(subset=['YEAR'])
batters_clean = batters_df.dropna(axis='columns')
cleaner_bats = batters_clean[batters_clean.YEAR != 'Total']
cleanest_bats = cleaner_bats[['espnID', 'playerName', 'YEAR', 'TEAM', 'WAR', '2B', '3B', 'AB', 'AVG', 'BB', 'CS', 'GP', 'H', 'HR', 'OBP', 'OPS', 'R',
       'RBI', 'SB', 'SLG', 'SO' ]]
cleanest_bats.head(10)

,espnID,playerName,YEAR,TEAM,WAR,2B,3B,AB,AVG,BB,...,GP,H,HR,OBP,OPS,R,RBI,SB,SLG,SO
2,33912,Cody Bellinger,2017,LAD,4.2,26,4,480,.267,64,...,132,128,39,.352,.933,87,97,10,.581,146
3,33912,Cody Bellinger,2018,LAD,4.2,28,7,557,.260,69,...,162,145,25,.343,.813,84,76,14,.470,151
4,33912,Cody Bellinger,2019,LAD,--,34,3,558,.305,95,...,156,170,47,.406,1.035,121,115,15,.629,108
2,34886,Alex Bregman,2016,HOU,1.8,13,3,201,.264,15,...,49,53,8,.313,.791,31,34,2,.478,52
3,34886,Alex Bregman,2017,HOU,4.1,39,5,556,.284,55,...,155,158,19,.352,.827,88,71,17,.475,97
4,34886,Alex Bregman,2018,HOU,6.9,51,1,594,.286,96,...,157,170,31,.394,.926,105,103,10,.532,85
5,34886,Alex Bregman,2019,HOU,--,37,2,554,.296,119,...,156,164,41,.423,1.015,122,112,5,.592,83
2,30836,Mike Trout,2011,LAA,0.6,6,0,123,.220,9,...,40,27,5,.281,.671,20,16,4,.390,30
3,30836,Mike Trout,2012,LAA,10.7,27,8,559,.326,67,...,139,182,30,.399,.963,129,83,49,.564,139
4,30836,Mike Trout,2013,LAA,9.2,39,9,589,.323,110,...,157,190,27,.432,.989,109,97,33,.557,136


In [112]:
pitchers_df = some_df.dropna(subset=['BLSV','CG'])
pitchers_clean = pitchers_df.dropna(axis='columns')
cleaner_pitch = pitchers_clean[pitchers_clean.SEASON != 'Total']
cleanest_pitch = cleaner_pitch[['espnID', 'playerName','SEASON','WAR', 'BB', 'BLSV', 'CG', 'ER', 'ERA', 'GP', 'GS', 'H', 'HLD', 'HR', 'IP',
       'L', 'R',  'SHO', 'SO', 'SV', 'TEAM', 'W', 'WHIP'
       ]]
cleanest_pitch.head(20)

,espnID,playerName,SEASON,WAR,BB,BLSV,CG,ER,ERA,GP,...,HR,IP,L,R,SHO,SO,SV,TEAM,W,WHIP
2,6341,Justin Verlander,2005,-0.2,5,0,0,9,7.15,2,...,1,11.1,2,9,0,7,0,DET,0,1.76
3,6341,Justin Verlander,2006,3.8,60,0,1,75,3.63,30,...,21,186.0,9,78,1,124,0,DET,17,1.33
4,6341,Justin Verlander,2007,3.9,67,0,1,82,3.66,32,...,20,201.2,6,88,1,183,0,DET,18,1.23
5,6341,Justin Verlander,2008,1.5,87,0,1,108,4.84,33,...,18,201.0,17,119,0,163,0,DET,11,1.40
6,6341,Justin Verlander,2009,5.2,63,0,3,92,3.45,35,...,20,240.0,9,99,1,269,0,DET,19,1.18
7,6341,Justin Verlander,2010,4.1,71,0,4,84,3.37,33,...,14,224.1,9,89,0,219,0,DET,18,1.16
8,6341,Justin Verlander,2011,8.3,57,0,4,67,2.40,34,...,24,251.0,5,73,2,250,0,DET,24,0.92
9,6341,Justin Verlander,2012,7.5,60,0,6,70,2.64,33,...,19,238.1,8,81,1,239,0,DET,17,1.06
10,6341,Justin Verlander,2013,4.6,75,0,0,84,3.46,34,...,19,218.1,12,94,0,217,0,DET,13,1.31
11,6341,Justin Verlander,2014,1.1,65,0,0,104,4.54,32,...,18,206.0,12,114,0,159,0,DET,15,1.40


In [113]:
# cleanest_bats.to_csv('batters_df.csv', index=False)
cleanest_pitch.to_csv('pitchers_df.csv', index=False)

# cleaner_bats.columns
# cleaner_pitch.columns

In [178]:
cleanest_pitch['playerName'].nunique()

54

In [193]:
# grouped = cleanest_pitch.groupby(['espnID','playerName','SEASON'])['WAR'].apply(list)
# jason = grouped.to_dict()
groupby = cleanest_pitch.groupby(['espnID','SEASON'])
x = groupby['playerName'].first()
w = groupby['WAR'].unique()
w.to_frame()

WAR
espnID SEASON                  
5203   2002               [0.9]
       2003               [1.0]
       2004               [1.8]
       2005               [4.0]
       2006               [4.3]
       2007               [6.0]
       2008               [3.3]
       2009               [1.6]
       2010               [1.5]
       2011              [-2.1]
       2013               [2.8]
       2014    [1.4, -0.3, 1.1]
       2015               [5.6]
       2016               [2.5]
       2017               [1.2]
5883   2004               [3.6]
       2005               [0.8]
       2006               [0.0]
       2007               [2.1]
       2008               [5.0]
       2009              [10.1]
       2010               [3.2]
       2011               [1.4]
       2012     [1.2, 2.1, 3.3]
       2013               [3.9]
       2014               [4.3]
       2015               [9.3]
       2016               [2.3]
       2017               [6.0]
       2018               [4.2]
...                         ...
33263  2019                [--]
33709  2015               [1.9]
       2016              [-0.0]
       2017               [4.5]
       2018              [10.5]
       2019                [--]
33748  2016               [0.8]
       2017               [1.0]
       2018               [7.5]
       2019                [--]
33837  2017              [-0.3]
       2018               [2.6]
       2019                [--]
33839  2017               [3.3]
       2018               [8.4]
       2019                [--]
34842  2016               [4.9]
       2017               [3.6]
       2018               [0.9]
34874  2016               [0.2]
       2017               [3.1]
       2018               [4.7]
       2019                [--]
34984  2018              [-0.0]
       2019                [--]
35124  2017               [2.5]
       2018               [1.6]
       2019                [--]
40912  2018               [1.0]
       2019                [--]

[430 rows x 1 columns]

In [132]:
# pprint(jason)

In [140]:
grouped2 = cleanest_pitch.groupby(['espnID','playerName'])['SEASON'].max()
jason2 = grouped2.to_dict()

In [141]:
pprint(jason2)

{(5203, 'John Lackey'): '2017',
 (5883, 'Zack Greinke'): '2019',
 (6216, 'Cole Hamels'): '2019',
 (6280, 'Ervin Santana'): '2019',
 (6341, 'Justin Verlander'): '2019',
 (28487, 'Jon Lester'): '2019',
 (28955, 'Johnny Cueto'): '2019',
 (28958, 'David Price'): '2019',
 (28962, 'Gio Gonzalez'): '2019',
 (28963, 'Clayton Kershaw'): '2019',
 (28968, 'Carlos Carrasco'): '2019',
 (28976, 'Max Scherzer'): '2019',
 (29155, 'Charlie Morton'): '2019',
 (29949, 'Madison Bumgarner'): '2019',
 (29966, 'Rick Porcello'): '2019',
 (30134, 'Andrew Cashner'): '2019',
 (30145, 'Jake Arrieta'): '2019',
 (30373, 'Stephen Strasburg'): '2019',
 (30624, 'Mike Minor'): '2019',
 (30820, 'Lance Lynn'): '2019',
 (30948, 'Chris Sale'): '2019',
 (30981, 'Corey Kluber'): '2019',
 (31091, 'Julio Teheran'): '2019',
 (31254, 'Aaron Sanchez'): '2019',
 (31258, 'Jameson Taillon'): '2019',
 (31313, 'Patrick Corbin'): '2019',
 (31340, 'Carlos Martinez'): '2019',
 (31593, 'Tanner Roark'): '2019',
 (31730, 'Noah Syndergaard')

In [60]:
#add name to dictionaries
for i in range(len(clean_data)):
    mongo_list[i]['name'] = clean_data['espn_name'][i]

In [61]:
#read in player photos list
%store -r player_photos

In [62]:
#add photo urls to dictionaries
for i in range(len(clean_data)):
    mongo_list[i]['photo_url'] = player_photos[i]

In [63]:
#store mongo list of entires
%store mongo_list
    



Stored 'mongo_list' (list)


In [59]:
mongo_list[0]

{'YEAR': 2     2017
 3     2018
 4     2019
 5    Total
 Name: YEAR, dtype: object, 'TEAM': 2      LAD
 3      LAD
 4      LAD
 5    Total
 Name: TEAM, dtype: object, 'GP': 2    132
 3    162
 4    156
 5    450
 Name: GP, dtype: object, 'AB': 2     480
 3     557
 4     558
 5    1595
 Name: AB, dtype: object, 'R': 2     87
 3     84
 4    121
 5    292
 Name: R, dtype: object, 'H': 2    128
 3    145
 4    170
 5    443
 Name: H, dtype: object, '2B': 2    26
 3    28
 4    34
 5    88
 Name: 2B, dtype: object, '3B': 2     4
 3     7
 4     3
 5    14
 Name: 3B, dtype: object, 'HR': 2     39
 3     25
 4     47
 5    111
 Name: HR, dtype: object, 'RBI': 2     97
 3     76
 4    115
 5    288
 Name: RBI, dtype: object, 'BB': 2     64
 3     69
 4     95
 5    228
 Name: BB, dtype: object, 'SO': 2    146
 3    151
 4    108
 5    405
 Name: SO, dtype: object, 'SB': 2    10
 3    14
 4    15
 5    39
 Name: SB, dtype: object, 'CS': 2    3
 3    1
 4    5
 5    9
 Name: CS, dtype: object,